In [1]:
import os
import re
import pandas as pd
import requests

In [ ]:
for file in os.listdir('./urls'):
    url = pd.read_csv('./urls/%s'%file)
    result = pd.read_csv('./data/%s'%file)
    url = url[~url['url'].isin(result['url'])]
    url.to_csv('./restdata/%s'%file)

In [ ]:
for file in os.listdir('./data'):
    pd.read_csv('./data/%s'%file).drop_duplicates(inplace=True).to_csv('.//%s'%file)

In [ ]:
path = os.listdir('./data')[7][:-4]

In [ ]:
data = pd.read_csv('./data/%s.csv'%path)

In [ ]:
data = data[data['成交时间']!='成交时间'].copy()

In [ ]:
def parse_func(s):
    try:
        if '-' in s:
            return s[:4]
    except:
        print(s)
    return s
data['成交年份'] = data['成交时间'].apply(parse_func)

In [ ]:
def parse_func(s):
    try:
        if '-' in s:
            return s.replace('-','年',1).replace('-','月',1)[:8]
    except:
        print(s)
    return s
data['成交年月'] = data['成交时间'].apply(parse_func)

In [ ]:
def parse_func(s):
    try:
        if '-' in s:
            return s.replace('-','/')
    except:
        print(s)
    return s
data['成交时间'] = data['成交时间'].apply(parse_func)

In [ ]:
def parse_func(s):
    try:
        s = re.findall('\d?\.?\d',s)[-1]
    except:
        pass
    return s
data['容积率(新)'] = data['容积率'].apply(parse_func)

In [ ]:
data['区县(新)'] = data['区县'].astype(str).str[:3]

In [ ]:
data.to_excel('./result/%s.xlsx'%path)

In [2]:
def gaodelocation(keyword):
    params = {
        "key": "d3c362e23d62acb3b586bad0f68bd826",
        "keywords": keyword,
        "types": 190301,
        'city': 410100,
        'citylimit':True,
#         'citylimit': True,
    }
    r = requests.get('https://restapi.amap.com/v3/place/text', params=params)
    if r.ok:
        result = r.json()
        try:
            s = ""
            for location in result['pois']:
                if location['name'] in keyword:
                    s += location['address']
            return s
        except:
            pass

def baidulocation(keyword):
    params = {
        "query": keyword,
#         "tag": '道路',
        'region': 268,
        'citylimit':True,
        'scope':1,
        'output':'json',
        'ak':"zLGTXliTc8proPtie1czOvX1VMXQM5TT",
    }
    r = requests.get('http://api.map.baidu.com/place/v2/search', params=params)
    if r.ok:
        result = r.json()
#         print(result)
        try:
            for location in result['results']:
                if location['name'] in keyword:
                    return location['area']
        except:
            pass

In [3]:
data = pd.read_excel('./result/住宅用地.xlsx')

In [4]:
data.loc[data['区县(新)']=='土地招','区县(新)'] = data.loc[data['区县(新)']=='土地招','宗地位置'].apply(gaodelocation)

In [5]:
data['起始价格（万元）'] = data['起始价（万元）'].astype(str).str.replace('万元','')
data['成交价格（万元）'] = data['成交价（万元）'].astype(str).str.replace('万元','')
data['出让年限'] = data['出让年限'].astype(str).str.replace('年','')

In [6]:
# def height(keyword):
#     try:
#         x = re.findall('\d+', keyword)[-1]
#         return x
#     except:
#         return keyword
# data['限高'] = data['限高（米）'].apply(height)

In [7]:
data.columns

Index(['Unnamed: 0', '用途', '宗地名称', '省市', '区县', '地块公告号', '宗地位置', '出让年限', '交易方式',
       '起始价（万元）', '成交价（万元）', '溢价率', '成交时间', '地块面积（平方米）', '建筑面积（平方米）', '限高（米）',
       '受让单位', '交易状况', 'url', '成交年份', '成交年月', '容积率(新)', '区县(新)', '起始价格（万元）',
       '成交价格（万元）'],
      dtype='object')

In [8]:
result = data[['用途','省市','区县','区县(新)','地块公告号','宗地名称','宗地位置','出让年限','交易方式','起始价格（万元）','成交价格（万元）','溢价率','成交年份','成交年月','成交时间','地块面积（平方米）','容积率','建筑面积（平方米）','限高（米）','受让单位','交易状况']]

KeyError: "['容积率'] not in index"

In [ ]:
result = result.replace('暂无','/')

In [28]:
result.to_excel('./resize/住宅用地.xlsx')

NameError: name 'result' is not defined

In [41]:
data = pd.read_excel('./resize/工业用地.xlsx')

In [37]:
data.drop(['列1'], axis=1, inplace=True)

In [44]:
data.loc[data['受让单位']=='/', '备注'] = '未成交'

In [39]:
data.loc[data['交易状况']=='已成交', '交易状况'] = None

In [57]:
data.to_excel('./resize/final/公建用地.xlsx')

In [60]:
data = pd.read_excel('./resize/final/公建用地.xlsx',col_index=0)

In [68]:
def fun(s):
    try:
        return re.findall('\d+', s)[-1]
    except:
        return s
data['限高（米）'] = data['限高（米）'].apply(fun)

In [69]:
data.to_excel('./resize/final/公建用地.xlsx')